In [1]:
# Data Loader
import os
import numpy as np
from random import shuffle
from skimage.io import imread
from skimage.transform import resize

In [2]:
def gen_labels(im_name, pat1, pat2):
    '''
    Parameters
    ----------
    im_name : Str
        The image file name.
    pat1 : Str
        A string pattern in the filename for 1st class, e.g "AFF"
    pat2 : Str
        A string pattern in the filename 2nd class, e.g, "Nff"
    Returns
    -------
    Label : Numpy array        
        Class label of the filename name based on its pattern.
    '''
    if pat1 in im_name:
        label = np.array([0])
    elif pat2 in im_name:
        label = np.array([1])
    return label


def get_data(data_path, data_list, img_h, img_w):
    """
    Parameters
    ----------
    train_data_path : Str
        Path to the data directory
    train_list : List
        A list containing the name of the images.
    img_h : Int
        image height to be resized to.
    img_w : Int
        image width to be resized to.    
    Returns
    -------
    img_labels : Nested List
        A nested list containing the loaded images along with their
        correcponding labels.
    """
    img_labels = []      
    for item in enumerate(data_list):
        img = imread(os.path.join(data_path, item[1]), as_gray = True) # "as_grey"
        img = resize(img, (img_h, img_w), anti_aliasing = True).astype('float32')
        img_labels.append([np.array(img), gen_labels(item[1], 'AFF', 'NFF')])
       
        if item[0] % 100 == 0:
             print('Reading: {0}/{1}  of train images'.format(item[0], len(data_list)))
             
    shuffle(img_labels)
    return img_labels


def get_data_arrays(nested_list, img_h, img_w):
    """
    Parameters
    ----------
    nested_list : nested list
        nested list of image arrays with corresponding class labels.
    img_h : Int
        Image height.
    img_w : Int
        Image width.
    Returns
    -------
    img_arrays : Numpy array
        4D Array with the size of (n_data,img_h,img_w, 1)
    label_arrays : Numpy array
        1D array with the size (n_data).
    """
    img_arrays = np.zeros((len(nested_list), img_h, img_w), dtype = np.float32)
    label_arrays = np.zeros((len(nested_list)), dtype = np.int32)
    for ind in range(len(nested_list)):
        img_arrays[ind] = nested_list[ind][0]
        label_arrays[ind] = nested_list[ind][1]
    img_arrays = np.expand_dims(img_arrays, axis =3)
    return img_arrays, label_arrays


def get_train_test_arrays(train_data_path, test_data_path, train_list,
                          test_list, img_h, img_w):
    """
    Get the directory to the train and test sets, the files names and
    the size of the image and return the image and label arrays for
    train and test sets.
    """
   
    train_data = get_data(train_data_path, train_list, img_h, img_w)
    test_data = get_data(test_data_path, test_list, img_h, img_w)
   
    train_img, train_label =  get_data_arrays(train_data, img_h, img_w)
    test_img, test_label = get_data_arrays(test_data, img_h, img_w)
    del(train_data)
    del(test_data)      
    return train_img, test_img, train_label, test_label

In [3]:
img_w, img_h = 128, 128 # Setting the width and heights of the images.
data_path = '/DL_course_data/Lab1/Bone/' # Path to data root with two subdirs.
train_data_path = os.path.join(data_path, 'train')
test_data_path = os.path.join(data_path, 'test')
train_list = os.listdir(train_data_path)
test_list = os.listdir(test_data_path)
x_train, x_test, y_train, y_test = get_train_test_arrays(
     train_data_path, test_data_path,
     train_list, test_list, img_h, img_w)

Reading: 0/1072  of train images
Reading: 100/1072  of train images
Reading: 200/1072  of train images
Reading: 300/1072  of train images
Reading: 400/1072  of train images
Reading: 500/1072  of train images
Reading: 600/1072  of train images
Reading: 700/1072  of train images
Reading: 800/1072  of train images
Reading: 900/1072  of train images
Reading: 1000/1072  of train images
Reading: 0/140  of train images
Reading: 100/140  of train images


In [4]:
from tensorflow.keras.layers import Input, Dense, Flatten 
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation
from tensorflow.keras.layers import Dropout 

In [5]:
def model(img_ch, img_width, img_height, n_base):
 
     model = Sequential()
 
     model.add(Conv2D(filters=n_base, input_shape=(img_width, img_height, img_ch),
                 kernel_size=(3,3), activation='relu', strides=(1,1), padding='same'))
     model.add(Conv2D(filters=n_base, input_shape=(img_width, img_height, img_ch),
                 kernel_size=(3,3), activation='relu', strides=(1,1), padding='same'))
     model.add(MaxPooling2D(pool_size=(2,2)))
    
     model.add(Conv2D(filters= n_base *2, kernel_size=(3,3), activation='relu', strides=(1,1), padding='same'))
     model.add(Conv2D(filters= n_base *2, kernel_size=(3,3), activation='relu', strides=(1,1), padding='same'))
     model.add(MaxPooling2D(pool_size=(2,2)))
        
     model.add(Conv2D(filters= n_base *4, kernel_size=(3,3), activation='relu', strides=(1,1), padding='same'))
     model.add(Conv2D(filters= n_base *4, kernel_size=(3,3), activation='relu', strides=(1,1), padding='same'))
     model.add(Conv2D(filters= n_base *4, kernel_size=(3,3), activation='relu', strides=(1,1), padding='same'))
     model.add(MaxPooling2D(pool_size=(2,2)))
        
     model.add(Conv2D(filters= n_base *8, kernel_size=(3,3), activation='relu', strides=(1,1), padding='same'))
     model.add(Conv2D(filters= n_base *8, kernel_size=(3,3), activation='relu', strides=(1,1), padding='same'))
     model.add(Conv2D(filters= n_base *8, kernel_size=(3,3), activation='relu', strides=(1,1), padding='same'))
     model.add(MaxPooling2D(pool_size=(2,2)))
        
     model.add(Conv2D(filters= n_base *8, kernel_size=(3,3), activation='relu', strides=(1,1), padding='same'))
     model.add(Conv2D(filters= n_base *8, kernel_size=(3,3), activation='relu', strides=(1,1), padding='same'))
     model.add(Conv2D(filters= n_base *8, kernel_size=(3,3), activation='relu', strides=(1,1), padding='same'))
     model.add(MaxPooling2D(pool_size=(2,2)))
        
     model.add(Flatten())
    
     model.add(Dense(64))
     model.add(Activation('relu'))
     model.add(Dropout(0.2))
        
     model.add(Dense(64))
     model.add(Activation('relu'))
     model.add(Dropout(0.2))   
    
     model.add(Dense(64))
     model.add(Activation('relu'))
     model.add(Dropout(0.2))
        
     model.add(Dense(1))
     model.add(Activation('sigmoid'))
     model.summary() 
     return model
firstlayer = 16
batchsize = 8
n_epochs = 100   # 20
learningrate = 0.00001   #0.00001

clf =  model(1,img_w, img_h,firstlayer)

clf.compile(loss='binary_crossentropy',
              optimizer = Adam(lr = learningrate),
              metrics=['binary_accuracy'])
clf_hist = clf.fit(x_train, y_train, epochs = n_epochs, batch_size = batchsize, validation_data = (x_test,y_test))

import matplotlib.pyplot as plt
plt.figure(figsize=(4, 4))
plt.title("Learning curve")
plt.plot(clf_hist.history["loss"], label="loss")
plt.plot(clf_hist.history["val_loss"], label="val_loss")
plt.plot( np.argmin(clf_hist.history["val_loss"]),
     np.min(clf_hist.history["val_loss"]),
     marker="x", color="r", label="best model")
plt.xlabel("Epochs")
plt.ylabel("Loss Value")
plt.legend();
plt.show()
print('the minimum var_loss is', np.min(clf_hist.history["val_loss"]))
print('the Best epoch nr is', np.argmin(clf_hist.history["val_loss"]))


#Accuracy
plt.figure(figsize=(4, 4))
plt.title("Accuracy")
plt.plot(clf_hist.history["binary_accuracy"], label="accuracy")
plt.plot(clf_hist.history["val_binary_accuracy"], label="val_accuracy")
xmax = np.argmax(clf_hist.history["val_binary_accuracy"])
ymax = np.max(clf_hist.history["val_binary_accuracy"])
plt.plot( xmax, ymax, marker="x", color="r", label="best model")
plt.annotate('(' + str(xmax) + ', '+ str(round(ymax,2)) + ')', xy = (xmax, ymax + 0.01),
             horizontalalignment = "center", verticalalignment = "bottom", color = "red")
plt.xlabel("Epochs")
plt.ylabel("Loss Value")
plt.legend();
print('the maximum var_accuracy is', np.max(clf_hist.history["val_binary_accuracy"]))
print('the maximum train_accuracy is', np.max(clf_hist.history["binary_accuracy"]))
print('the Best epoch nr is', xmax)



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 16)      160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 16)      2320      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 32)        4640      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 64)        1

Epoch 25/100
1072/1072 [==============================] - 2s 2ms/sample - loss: 0.4916 - binary_accuracy: 0.7556 - val_loss: 0.4260 - val_binary_accuracy: 0.7643
Epoch 26/100
1072/1072 [==============================] - 2s 2ms/sample - loss: 0.4471 - binary_accuracy: 0.7910 - val_loss: 0.4254 - val_binary_accuracy: 0.7500
Epoch 27/100
1072/1072 [==============================] - 2s 2ms/sample - loss: 0.4431 - binary_accuracy: 0.7929 - val_loss: 0.3896 - val_binary_accuracy: 0.7929
Epoch 28/100
1072/1072 [==============================] - 2s 2ms/sample - loss: 0.4270 - binary_accuracy: 0.7929 - val_loss: 0.3688 - val_binary_accuracy: 0.7929
Epoch 29/100
1072/1072 [==============================] - 2s 2ms/sample - loss: 0.4030 - binary_accuracy: 0.8060 - val_loss: 0.4055 - val_binary_accuracy: 0.7786
Epoch 30/100
1072/1072 [==============================] - 2s 2ms/sample - loss: 0.4011 - binary_accuracy: 0.8013 - val_loss: 0.3847 - val_binary_accuracy: 0.7857
Epoch 31/100
1072/1072 [====

1072/1072 [==============================] - 2s 2ms/sample - loss: 0.2259 - binary_accuracy: 0.9049 - val_loss: 0.3078 - val_binary_accuracy: 0.8357
Epoch 76/100
1072/1072 [==============================] - 2s 2ms/sample - loss: 0.2247 - binary_accuracy: 0.8965 - val_loss: 0.5571 - val_binary_accuracy: 0.8000
Epoch 77/100
1072/1072 [==============================] - 2s 2ms/sample - loss: 0.2254 - binary_accuracy: 0.9021 - val_loss: 0.3411 - val_binary_accuracy: 0.8214
Epoch 78/100
1072/1072 [==============================] - 2s 2ms/sample - loss: 0.2139 - binary_accuracy: 0.8955 - val_loss: 0.3267 - val_binary_accuracy: 0.8357
Epoch 79/100
1072/1072 [==============================] - 2s 2ms/sample - loss: 0.2193 - binary_accuracy: 0.9039 - val_loss: 0.3294 - val_binary_accuracy: 0.8214
Epoch 80/100
1072/1072 [==============================] - 2s 2ms/sample - loss: 0.2036 - binary_accuracy: 0.9170 - val_loss: 0.3268 - val_binary_accuracy: 0.8357
Epoch 81/100
1072/1072 [=================

<Figure size 400x400 with 1 Axes>

the minimum var_loss is 0.2792458317375609
the Best epoch nr is 90
the maximum var_accuracy is 0.85
the maximum train_accuracy is 0.9375
the Best epoch nr is 55


In [6]:
x_train.shape

(1072, 128, 128, 1)

In [7]:
x_test.shape

(140, 128, 128, 1)